<a href="https://colab.research.google.com/github/jorgealarconlucas/TFG/blob/master/Modelos_de_ML_con_tendencia_temporal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Machine learning añadiendo información sobre la tendencia temporal

In [106]:
import pandas as pd
import numpy as np
import statistics as stats
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt
import scipy.stats as ss
import statistics as stats
import seaborn as sns 
from pandas.plotting import autocorrelation_plot
from sklearn.model_selection import train_test_split
from sklearn.tree import export_graphviz
from six import StringIO
from IPython.display import Image  
import pydotplus
from sklearn.model_selection import  GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,recall_score, precision_score, confusion_matrix
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score

In [107]:
#db = pd.read_csv('/content/Base datos COVID-Linfos Actualizazda.xlsx - Valores seleccionados.csv')
db = pd.read_excel('Base datos COVID-Linfos Actualizazda.xlsx') #no quites esta línea, coméntala cuando estés trabajando en tus datos

db.head(8)

,Número,Repetición NHC,Código Externo,FALLECE,UCI,Día del Año,19nCv,Día de POSITIVO,Días respecto a Positivo,Fecha,...,ESSYS,ASLYP,ASLYA,RELPL,RELYP,RELYA,NEUGI,NEURI,ASLPL,19nCv.1
0,221.0,4,111561,NaN,NaN,90,NaN,NaN,NaN,30/03/20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,319.0,13,102441,NaN,NaN,100,NaN,NaN,NaN,9/04/20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,368.0,16,102441,NaN,NaN,104,NaN,NaN,NaN,13/04/20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1759.0,9,104851,NaN,NaN,238,NaN,NaN,NaN,25/08/20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,265277.0,7,11258,NaN,NaN,91,NaN,NaN,NaN,31/03/20,...,NaN,0,0,5.7,2.3,0.13,154.8,50.2,0,NaN
5,325932.0,11,115555,NaN,NaN,98,NaN,NaN,NaN,7/04/20,...,NaN,0,0,6.6,2.1,0.15,147.1,47.8,0,NaN
6,366742.0,5,111520,NaN,NaN,83,NaN,NaN,NaN,23/03/20,...,NaN,0,0,5.4,1.8,0.13,152.6,49.9,0,NaN
7,374616.0,6,116651,NaN,NaN,97,NaN,NaN,NaN,6/04/20,...,NaN,0,0,11.5,5,0.25,150.0,52.7,0,NaN


### 1.1 Eliminación de columnas y filas (consultado con Javier)

Tras una visualización general y una charla con los sanitarios que han transferido la base de datos, se llega a la conclusión de que hay que hacer una limpieza de la misma. Hay columnas que su significado y valores no tienen ninguna relevancia para el estudio que se va a llevar a cabo.

In [108]:
db1=db.drop(["Repetición NHC","Día del Año","Día de POSITIVO","Días respecto a Positivo","GSRH","GRHPC","GRHRN","GRHCE","FRSP","COMHE","AVSBL","AVSPL","AVSRJ","AVNRB","ESSYS"],axis=1) #axis=1 ya que lo que elimino son columnas.


A continuación elimino las filas que no aportan información.

In [109]:
db1= db1.dropna(thresh=9) #Mantiene solo las filas con al menos 9 valores que no sean NA. Esto me sirve ya que quiero eliminar aquellas filas
#que tengan NaN, es decir 19nCv y todos los parámetros del hemograma.
db1.shape

(320, 122)

Además teniendo en cuenta la informacion proporcionada por parte del hospital, elimino aquellas variables que son redundantes. Estas variables no aportan información adicional al futuro modelo de predicción.

In [110]:
db1=db1.drop(["NE_R","NEU_a","NEU_p","LY_R","LIN_a","LIN_p","MO_R","EO_R","BA_R"],axis=1) #axis=1 ya que lo que elimino son columnas.

### 1.2 Eliminación de columnas con alta correlación

Tras comprobar las variables con alta correlación en el dendrograma se llega a la conclusión junto con el médico Javier, de eliminar una serie de variables por ser redundantes. Los grupos de parámetros sobre los que se discute son: (ERBLp, ERBL) (NEURI, NESFL) (NEUGI, NESSC) (PLTI, PLT) (WBC-N, LEU, TNC-N, WBC-D) (HCT, HGB) (ASLYP, ASLYA) (EOSP, EOS):

In [111]:
db1=db1.drop(["ERBLp","NESFL","NESSC","PLT","WBC-N","TNC-N","WBC-D","BASp","HCT","ASLYA","EOSp"],axis=1)

### 1.3 Agrupación de pacientes por gravedad


- UCI=1 y FALLECE=1 --> Muy grave
- UCI=0 y FALLLECE=1 --> Muy grave

- UCI=1 y FALLECE=0 --> Grave
- UCI=0 y FALLECE=0 --> Sano

In [112]:
data_all = db1[(db1['UCI'] == 1.0) & (db1['FALLECE'] == 1.0)]
data_death = db1[(db1['UCI'] == 0.0) & (db1['FALLECE'] == 1.0)]
data_regular = db1[(db1['UCI'] == 1.0) & (db1['FALLECE'] == 0.0)]
data_health = db1[(db1['UCI'] == 0.0) & (db1['FALLECE'] == 0.0)]

### 1.4 Tipo de cada columna

In [113]:
db1.info(verbose=True, null_counts=True) #Hay un total de 342 filas

<class 'pandas.core.frame.DataFrame'>
Int64Index: 320 entries, 4 to 341
Data columns (total 102 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Número             319 non-null    float64
 1   Código Externo     320 non-null    int64  
 2   FALLECE            96 non-null     float64
 3   UCI                96 non-null     float64
 4   19nCv              105 non-null    object 
 5   Fecha              320 non-null    object 
 6   Edad del Paciente  320 non-null    object 
 7   Sexo del Paciente  320 non-null    object 
 8   LEU                284 non-null    float64
 9   NEU                284 non-null    float64
 10  NEUp               284 non-null    float64
 11  LIN                284 non-null    float64
 12  LINp               284 non-null    float64
 13  MON                284 non-null    float64
 14  MONp               284 non-null    float64
 15  EOS                284 non-null    float64
 16  BAS                284 no

In [114]:
#Cambio de comas por puntos
db1 = db1.replace(',','.', regex=True)

#Reemplazo esos caracteres por NaN
db1=db1.replace("----",np.nan)

# Función que elimina la palabra años
def only_digits (txt):
  return "".join(c for c in txt if c.isdigit())

db1["Edad del Paciente"] = db1["Edad del Paciente"].map(only_digits)

## MUY IMPORTANTE. Conversión de variables excepto 19nCv, Fecha, Sexo del Paciente y 19nCv.1 
# Una vez hecho el cambio de comas por puntos y haber introducido NaN por "----", este código me transforma los 'object' en 'numerics'
for i in db1:
  if not (i=="19nCv" or i=="Fecha" or i=="Sexo del Paciente" or i=="19nCv.1"):
    if db1[i].dtypes == object:
      db1[i] = pd.to_numeric(db1[i])

#Paso la variable "fecha" de tipo object a tipo datatime, para poder tratar con ella como una fecha.
db1["Fecha"]=pd.to_datetime(db1["Fecha"])

## 1.5 Intruducción de nuevas columnas, "Revision" y "Label"


In [115]:
Codigo_lista = db1['Código Externo'].tolist()

In [116]:
from collections import Counter

lista_pacientes = Counter(Codigo_lista).keys()

### INICIALIZO DATAFRAME DE SANOS
sanos_df = pd.DataFrame()

### INICIALIZO DATAFRAME DE NO SANOS
no_sanos_df = pd.DataFrame()

for i in lista_pacientes:  
      if i not in data_health.values:
        new_1 = db1.loc[db1["Código Externo"] == i]
        new_1 = new_1.sort_values('Fecha') #ordenar de la revision mas antigua a la mas reciente.
        new_1['Revision'] = range(1, 1+len(new_1)) #en la nueva columna "Revision" se van a añadiendo valores de manera incremental desde 1 teniendo 
                                          # en cuenta el número de citas de cada paciente.

        new_1 = new_1.assign(label=1) #Introduzco nueva columna con etiqueta 1 (NO SANO)

        no_sanos_df = no_sanos_df.append(new_1)

      else:
        new_2 = db1.loc[db1["Código Externo"] == i]
        new_2 = new_2.sort_values('Fecha')
        new_2['Revision'] = range(1, 1+len(new_2)) #en la nueva columna "Revision" se van a añadiendo valores de manera incremental desde 1 teniendo 
                                          # en cuenta el número de citas de cada paciente.

        new_2 = new_2.assign(label=0)#Introduzco nueva columna con etiqueta 0 (SANO)
      
        sanos_df = sanos_df.append(new_2)


# 2. Preprocesado de datos

### ¿Hasta que revisión se cuenta con un número adecuado de pacientes?

Se realiza el proyecto con un total de 4 revisiones. Se llega a esta conclusión de la siguiente forma; se dividen las variables de forma individual y se va representando cada uno de los valores de cada variable en cada revisión (teniendo así una visión de los valores de la variable para todos los pacientes en cada revisión a medida que pasa el tiempo). Esto se realiza con los pacientes sanos por un lado y los pacientes no sanos por el otro. Finalmente, se observa como con 4 revisiones se puede hacer un estudio amplio y completo de las variables. 

Además, de 98 pacientes, 44 pacientes presentan 4 revisiones, es decir el 44,89% de los pacientes presentes en la base de datos, porcentaje considerable que ayuda a tomar la decision de usar 4 revisiones.


In [117]:
db1 = pd.concat([sanos_df, no_sanos_df])
db1.shape

print(db1.columns)

Index(['Número', 'Código Externo', 'FALLECE', 'UCI', '19nCv', 'Fecha',
       'Edad del Paciente', 'Sexo del Paciente', 'LEU', 'NEU',
       ...
       'ASLYP', 'RELPL', 'RELYP', 'RELYA', 'NEUGI', 'NEURI', 'ASLPL',
       '19nCv.1', 'Revision', 'label'],
      dtype='object', length=104)


## 2.1 Identificacion de valores perdidos

Junto con el estudio del tipo de variables, es básico conocer el número de observaciones disponibles y si todas ellas están completas. Los valores perdidos son muy importantes a la hora de crear modelos, la mayoría de algoritmos no aceptan observaciones incompletas o bien se ven muy influenciados por ellas.

In [118]:
# Columnas con missing values

for index,value in enumerate (db1.isnull().sum()):
  if value !=0:
    print(db1.columns[index],value)

Número 1
FALLECE 224
UCI 224
19nCv 215
LEU 36
NEU 36
NEUp 36
LIN 36
LINp 36
MON 36
MONp 36
EOS 36
BAS 36
IG 36
IGp 36
ERIT 36
HGB 36
VCM 36
HCM 36
CHCM 36
RDW 36
ERBL 36
RET 311
RETp 311
FRI 311
RETHE 311
PLTI 38
PLTF 316
PLTO 311
VPM 43
IPF 316
MNEU 313
MLIN 313
MMON 313
MBAS 313
MEOS 313
MCAS 313
MMET 317
MMIE 318
MPRO 318
CPLAS 318
MBLS 318
MERI 317
rNe/L 160
rPL/L 160
HIPER 311
HYPO 311
MACR 320
MICRO 320
FRAGM 311
MacR 36
MicR 36
RBCO 320
MCHCO 311
BA-D# 36
NEFSC 36
BA-D% 36
BA-N# 36
BA-N% 36
D-HE 311
FRCA 311
HFLCA 36
HFLCP 36
HIPF 316
IPFA 316
IRF-Y 311
LY-WX 36
LY-WY 36
LY-WZ 36
LY-X 36
LY-Y 36
LY-Z 36
MO-WX 36
MO-WY 36
MO-WZ 36
MO-X 36
MO-Y 36
MO-Z 36
NE-WX 36
NE-WY 36
NE-WZ 36
RBCHE 311
RBC-Y 311
RTNC 311
RUPP 311
RET-Y 311
RPI 311
TNC-D 320
TNC-P 320
WBC-P 320
ASLYP 42
RELPL 42
RELYP 42
RELYA 42
NEUGI 36
NEURI 36
ASLPL 42
19nCv.1 269


In [119]:
#Quitar todas las columnas con mas de 310 NaN

#320-310 = 10. Las columnas con 311 NaN tienen 9 valores no NaN. Por lo que si me quedo con aquellas columnas que tengan como mínimo 10 valores
# no NaN, elimino todas las columnas con 333 NaN o más.
db1=db1.dropna(thresh=10,axis=1) 

db1=db1.drop(["Número","19nCv","19nCv.1", "FALLECE", "UCI", "Fecha"],axis=1)


print(db1.columns)

Index(['Código Externo', 'Edad del Paciente', 'Sexo del Paciente', 'LEU',
       'NEU', 'NEUp', 'LIN', 'LINp', 'MON', 'MONp', 'EOS', 'BAS', 'IG', 'IGp',
       'ERIT', 'HGB', 'VCM', 'HCM', 'CHCM', 'RDW', 'ERBL', 'PLTI', 'VPM',
       'rNe/L', 'rPL/L', 'MacR', 'MicR', 'BA-D#', 'NEFSC', 'BA-D%', 'BA-N#',
       'BA-N%', 'HFLCA', 'HFLCP', 'LY-WX', 'LY-WY', 'LY-WZ', 'LY-X', 'LY-Y',
       'LY-Z', 'MO-WX', 'MO-WY', 'MO-WZ', 'MO-X', 'MO-Y', 'MO-Z', 'NE-WX',
       'NE-WY', 'NE-WZ', 'ASLYP', 'RELPL', 'RELYP', 'RELYA', 'NEUGI', 'NEURI',
       'ASLPL', 'Revision', 'label'],
      dtype='object')


A continuación se hace un tratamiento de la variable categorica, para facilitar el trabajo con ella.

In [120]:
# Se convierte la variable Sexo del paciente a variable categórica

db1['Sexo del Paciente'] = db1['Sexo del Paciente'].astype('category').cat.codes # 0 --> Femenino, 1--> Masculino

In [121]:
db1

,Código Externo,Edad del Paciente,Sexo del Paciente,LEU,NEU,NEUp,LIN,LINp,MON,MONp,...,NE-WZ,ASLYP,RELPL,RELYP,RELYA,NEUGI,NEURI,ASLPL,Revision,label
122,11258,65,1,5.45,3.74,68.7,1.07,19.6,0.48,8.8,...,710.0,1.5,15.9,3.1,0.17,155.1,53.9,7.5,1,0
152,11258,65,1,5.01,2.68,53.5,1.42,28.3,0.58,11.6,...,569.0,0.0,16.2,4.6,0.23,149.9,49.8,0.0,2,0
4,11258,65,1,5.57,2.55,45.8,2.27,40.8,0.47,8.4,...,521.0,0.0,5.7,2.3,0.13,154.8,50.2,0.0,3,0
180,115555,69,1,10.38,8.65,83.3,0.88,8.5,0.78,7.5,...,694.0,0.4,12.5,1.1,0.11,143.9,48.3,4.5,1,0
124,115555,69,1,7.11,3.61,50.8,2.85,40.1,0.62,8.7,...,739.0,0.0,3.2,1.3,0.09,145.3,48.4,0.0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,114819,69,0,13.67,8.29,60.7,2.08,15.2,2.54,18.6,...,906.0,1.4,13.9,2.1,0.29,131.2,35.4,9.1,4,1
328,116453,66,1,11.14,9.94,89.3,0.64,5.7,0.18,1.6,...,794.0,0.5,17.2,1.0,0.11,153.2,49.9,9.4,1,1
332,116453,66,1,15.00,11.28,75.1,1.50,10.0,0.97,6.5,...,600.0,0.3,7.3,0.7,0.11,160.2,48.2,2.7,2,1
336,116453,66,1,11.49,10.30,89.6,0.49,4.3,0.45,3.9,...,712.0,0.0,10.2,0.4,0.05,160.0,50.6,0.0,3,1


## 2.2 Separación en train y test

Ahora debo comenzar con la división de train y test. Hay que tener en cuenta, que a la hora de dividir no me convendría tener la revisión 1 y 3 de un paciente en train y la revisión 2 del mismo paciente en test, por lo que eso debo de tenerlo MUY EN CUENTA

Por lo tanto, la división hay que hacerla por código externo.

Los pasos a seguir para llevar a cabo dicha división serán:

1. Extraer el código externo

2. Eliminar las duplicidades consiguiendo de esa manera un vector de valores únicos de la columna "Código Externo"

3. Shuffle del vector creado

4. Teniendo en cuenta que para train me quedo con el 80% de los datos y para test con el 20% restante, de los 98 pacientes que tengo, me quedaré con 78 para train y 20 para test

In [122]:
# Codigo_lista muestra todos los códigos externos y ha sido creada a partir de convertir a lista la columna 
# "Código Externo"

cd_unique = np.unique(Codigo_lista)
print("La lista de códigos externos únicos es:",cd_unique)

La lista de códigos externos únicos es: [  1164   1214  10074  10092  11197  11258  11372  12016  13299  13397
 101480 101609 101660 102441 104431 104798 104849 104851 105140 105582
 105639 106066 107746 107747 108527 108894 108897 109036 109115 109129
 109339 109887 110011 110435 110607 111310 111339 111520 111561 111786
 112135 112511 112823 112894 114183 114495 114819 115555 115784 115847
 116005 116033 116159 116453 116651 116806 117212 117398 117462 118023
 118401 119606 119737 119766 120418 121289 121358 122654 122958 123156
 123456 123676 123724 124738 124850 126455 126638 127383 128124 128148
 128864 130481 130937 131267 131394 132015 132432 133617 133978 134492
 134855 135062 135217 136647 136922 137069 137304 138076]


In [145]:
import random

#Desordeno de manera aleatoria la lista de códigos externos
random.seed(10)
random.shuffle(cd_unique)

#Hago la división en train y test, quedándome con los 78 primeros valores de la lista para train y los 20
#valores restantes para test

cd_train = cd_unique[0:78]
cd_test = cd_unique[78:98]

Debo crear un dataframe donde estén todas las revisiones de los primeros 78 códigos externos seleccionados anteriormente. A partir de este dataframe sacaré un x_train y un y_train que será "label".

In [146]:
db1.columns

Index(['Código Externo', 'Edad del Paciente', 'Sexo del Paciente', 'LEU',
       'NEU', 'NEUp', 'LIN', 'LINp', 'MON', 'MONp', 'EOS', 'BAS', 'IG', 'IGp',
       'ERIT', 'HGB', 'VCM', 'HCM', 'CHCM', 'RDW', 'ERBL', 'PLTI', 'VPM',
       'rNe/L', 'rPL/L', 'MacR', 'MicR', 'BA-D#', 'NEFSC', 'BA-D%', 'BA-N#',
       'BA-N%', 'HFLCA', 'HFLCP', 'LY-WX', 'LY-WY', 'LY-WZ', 'LY-X', 'LY-Y',
       'LY-Z', 'MO-WX', 'MO-WY', 'MO-WZ', 'MO-X', 'MO-Y', 'MO-Z', 'NE-WX',
       'NE-WY', 'NE-WZ', 'ASLYP', 'RELPL', 'RELYP', 'RELYA', 'NEUGI', 'NEURI',
       'ASLPL', 'Revision', 'label'],
      dtype='object')

In [147]:
### INICIALIZO DATAFRAME DE TRAINING
training = pd.DataFrame()

for i in cd_train:  
      pt_train = db1.loc[db1["Código Externo"] == i]

      training = training.append(pt_train)


### INICIALIZO DATAFRAME DE TEST
test = pd.DataFrame()

for i in cd_test:  
      pt_test = db1.loc[db1["Código Externo"] == i]

      test = test.append(pt_test)

In [148]:
test.shape

(60, 58)

In [149]:
training.shape

(260, 58)

A partir de los dataframes que he obtenido, de "training" extraigo x_train (todos los parámetros menos 'label) e y_train (el parámetro 'label). Con el dataframe 'test' sigo el mismo procedimiento, de tal manera que acabo obteniedno x_test e y_test

In [150]:
#esto está mal porque ya se han eliminado variables y hay menos de 61

#DIVISION EN TRAIN Y TEST
x_train = training.iloc[:,0:57]
y_train = training.iloc[:, 57]

x_test = test.iloc[:,0:57]
y_test = test.iloc[:, 57]

In [151]:
x_train.columns
x_train.shape

(260, 57)

## 2.3 Identificación de outliers e imputacion de los mismos en X_train usando el método KNN Imputer

Para detectar los outliers el primer paso es realizar una inspección visual a través de los boxplots y localizar aquellos que sobresalen del rango normal de valores.

In [152]:
x_train.plot(kind='box',subplots=True, layout=(21,6), sharex=False, sharey=False, figsize=(30,70));

A continuación calculo el z-score. Cada puntuación z nos dice a cuántas desviaciones estándar se encuentra un valor individual de la media de los valores del conjunto que estoy estudiando.

Se calcula el z-score sin normalizar y comprobar si cada valor se encuentra dentro de 3 veces la desviación típica. En el caso de que se encuentren fuera de ese intervalo se le considera outlier y será reemplazado a NaN.

In [153]:
def outliers (datos,dtype): #Función z-score
    count=0
    lista=[]
    for i in dtype[datos]:
        if not (dtype[datos].mean()-3*dtype[datos].std())<=i<=(dtype[datos].mean()+3*dtype[datos].std()): # 3 veces la desviacion estandar
            if not np.isnan(i):
                count=count+1
                lista.append(i)
   
    return lista 

In [154]:
numeric_variables_train = x_train.select_dtypes(exclude=['object'])

x_train_aux = x_train.copy()

for i in numeric_variables_train:
   x_train_aux[[i]]=x_train[[i]].replace(outliers(i,x_train),np.nan) 

In [156]:
x_train_aux.shape

(260, 57)

In [157]:
for index,value in enumerate (x_train.isnull().sum()):
  if value !=0:
    print(x_train_aux.columns[index],value)

LEU 29
NEU 29
NEUp 29
LIN 29
LINp 29
MON 29
MONp 29
EOS 29
BAS 29
IG 29
IGp 29
ERIT 29
HGB 29
VCM 29
HCM 29
CHCM 29
RDW 29
ERBL 29
PLTI 30
VPM 33
rNe/L 130
rPL/L 130
MacR 29
MicR 29
BA-D# 29
NEFSC 29
BA-D% 29
BA-N# 29
BA-N% 29
HFLCA 29
HFLCP 29
LY-WX 29
LY-WY 29
LY-WZ 29
LY-X 29
LY-Y 29
LY-Z 29
MO-WX 29
MO-WY 29
MO-WZ 29
MO-X 29
MO-Y 29
MO-Z 29
NE-WX 29
NE-WY 29
NE-WZ 29
ASLYP 33
RELPL 33
RELYP 33
RELYA 33
NEUGI 29
NEURI 29
ASLPL 33


In [159]:
x_train_aux.columns

Index(['Código Externo', 'Edad del Paciente', 'Sexo del Paciente', 'LEU',
       'NEU', 'NEUp', 'LIN', 'LINp', 'MON', 'MONp', 'EOS', 'BAS', 'IG', 'IGp',
       'ERIT', 'HGB', 'VCM', 'HCM', 'CHCM', 'RDW', 'ERBL', 'PLTI', 'VPM',
       'rNe/L', 'rPL/L', 'MacR', 'MicR', 'BA-D#', 'NEFSC', 'BA-D%', 'BA-N#',
       'BA-N%', 'HFLCA', 'HFLCP', 'LY-WX', 'LY-WY', 'LY-WZ', 'LY-X', 'LY-Y',
       'LY-Z', 'MO-WX', 'MO-WY', 'MO-WZ', 'MO-X', 'MO-Y', 'MO-Z', 'NE-WX',
       'NE-WY', 'NE-WZ', 'ASLYP', 'RELPL', 'RELYP', 'RELYA', 'NEUGI', 'NEURI',
       'ASLPL', 'Revision'],
      dtype='object')

***Imputacion de valores perdidos mediante KNN Imputer***

A continuación haré uso de KNNImputer. KNNImputer es una clase de scikit-learn que se utiliza para completar o predecir los valores faltantes en un conjunto de datos. Es un método que funciona con el enfoque básico del algoritmo KNN y que se considera que proporciona una mejor rendimiento que otros enfoques, como puedes ser la imputación de los outliers por la media o la mediana.

El siguiente código tiene como fin, ayudar en la elección del número K óptimo con el que realizar la imputación de los outliers convertidos en NaN. La investigación sugiere que es imperativo probar el modelo usando validación cruzada después de realizar la imputación con diferentes valores de k. Aunque la imputación de valores perdidos es un campo de estudio en constante evolución, kNN actúa como una estrategia simple y efectiva.

Al final de la ejecución, se crea un diagrama de caja y bigotes para cada conjunto de resultados, lo que permite comparar la distribución de los valores finales.

In [160]:
x_train_aux.shape

(260, 57)

>1 0.894 (0.037)
>3 0.894 (0.037)
>5 0.895 (0.033)
>7 0.885 (0.033)
>9 0.882 (0.034)
>15 0.886 (0.030)
>18 0.886 (0.030)
>21 0.874 (0.032)
>25 0.873 (0.030)
>30 0.880 (0.035)
>36 0.881 (0.032)
>40 0.881 (0.035)


In [161]:
from numpy import mean
from numpy import std
from pandas import read_csv
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import KNNImputer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from matplotlib import pyplot

# evaluate each strategy on the dataset
results = list()
strategies = [str(i) for i in [1,3,5,7,9,15,18,21,25,30,36,40]]
for s in strategies:
    # create the modeling pipeline
    
    #me parece ok usar RF
    pipeline = Pipeline(steps=[('i', KNNImputer(n_neighbors=int(s))), ('m', RandomForestClassifier())])  ###NO SE QUE ALGORITMO UTILIZAR
    # evaluate the model
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=1)  ###NO SE QUE NUMEROS PONER EN LOS PARAMETROS
    #Pongo X_train e y_train, ya que son los conjuntos sobre los que voy a trabajar
    scores = cross_val_score(pipeline, x_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1) ###NO SE QUE X E Y PONER
    # store results
    results.append(scores)
    print('>%s %.3f (%.3f)' % (s, mean(scores), std(scores)))
# plot model performance for comparison
pyplot.boxplot(results, labels=strategies, showmeans=True)
pyplot.show()

En este caso, podemos ver que los valores de k no siguen una uniformidad a la hora de dar un rendimiento óptimo del algoritmo en cuestión. Por lo tanto, se puede observar que para un k = --- el modelo presenta el mejor rendimiento con una precisión de alrededor de --- por ciento.


**Óscar**

Si te fijas sólo en las medias (los triangulitos, se ve que enseguida saturan y se ponen a un buen nivel, así que lo que yo haría sería quderme con un ńumero relativamente pqueño, por ejemplo 5)

Por esa razón, utilizaremos dicho valor de K en KNNImputer. Además, tendremos en cuenta otros 2 parámetros. 



*   Por un lado, se tiene en cuenta la medida de distancia, la cual se puede ponderar proporcionalmente a la distancia entre instancias (filas), aunque esto se establece en una ponderación uniforme por defecto, controlada a través del argumento "weights". En este caso, se opta por darle un valor unirforme, por lo que todas las muestras tienen la misma importancia a la hora de "influenciar" a la muestra NaN.

*   Por otro lado, se encuentra el parámetro "metric", el cual hace referencia a la medida de distancia que se utilizará para calcular la distancia entre los miembros del conjunto de datos de entrenamiento. Esta vez se opta por la distancia euclidea, ya que es la distancia utilizada en algunos de los principales métodos para ajustar estimaciones estadísticas de los datos minimizando errores, como el método de mínimos cuadrados.



In [162]:
from sklearn.impute import KNNImputer
from numpy import isnan

# print total missing
print('Total missing values before imputation: %d' % x_train_aux.isnull().sum().sum())

# define imputer
imputer = KNNImputer(n_neighbors=5, metric='nan_euclidean', weights='uniform') #Se calcula la distancia euclidea del punto NaN a los n vecinos 
                                      # mas cercanos. Además se le da un peso uniforme a todos los vecinos, es decir todos los vecinos tienen
                                      # la misma importancia.

# fit on the dataset
imputer.fit(x_train_aux)

# transform the dataset
x_train_aux = imputer.transform(x_train_aux)

# print total missing
print('Total missing values after imputation: %d' % sum(isnan(x_train_aux).flatten()))

Total missing values before imputation: 1968
Total missing values after imputation: 0


In [163]:
x_train_aux.shape

(260, 57)

In [164]:
x_train_aux = pd.DataFrame(x_train_aux) #paso X_train de numpy_array a DataFrame

#Compruebo que no queda ningún NaN en la base de datos. De esa manera ya puedo trabajar con ella.
check_for_nan = x_train.isnull().values.any()
print (check_for_nan)




True


In [168]:
x_train_aux.columns = x_train.columns

## 2.4 Identificación de outliers e imputacion de los mismos en X_test usando el método KNN Imputer




In [169]:
numeric_variables_test = x_test.select_dtypes(exclude=['object'])

for i in numeric_variables_test:
   x_test[[i]]=x_test[[i]].replace(outliers(i,x_test),np.nan) 

In [170]:
for index,value in enumerate (x_test.isnull().sum()):
  if value !=0:
    print(x_test.columns[index],value)

LEU 7
NEU 9
NEUp 7
LIN 9
LINp 8
MON 7
MONp 8
EOS 9
BAS 7
IG 8
IGp 8
ERIT 7
HGB 7
VCM 9
HCM 9
CHCM 7
RDW 7
ERBL 9
PLTI 8
VPM 11
rNe/L 31
rPL/L 31
MacR 7
MicR 9
BA-D# 7
NEFSC 7
BA-D% 8
BA-N# 7
BA-N% 8
HFLCA 9
HFLCP 7
LY-WX 7
LY-WY 8
LY-WZ 8
LY-X 7
LY-Y 7
LY-Z 7
MO-WX 8
MO-WY 8
MO-WZ 7
MO-X 7
MO-Y 7
MO-Z 7
NE-WX 8
NE-WY 8
NE-WZ 7
ASLYP 10
RELPL 10
RELYP 9
RELYA 10
NEUGI 7
NEURI 7
ASLPL 11


In [171]:
# print total missing
print('Total missing values before imputation: %d' % x_test.isnull().sum().sum())

#OJO: yo diría que esto es una mala práctica, no es malísima, pero no estás siendo justo. Lo ideal es, ahora
#que has entrenado el model en training bien (el KNNImputer), utilizar ese entrenamiento con los nuevos datos, no 
#volver a ajustar los datos. 

# define imputer
#imputer = KNNImputer(n_neighbors=21, weights='uniform', metric='nan_euclidean')


# fit on the dataset
# imputer.fit(X_test)  

# transform the dataset
x_test = imputer.transform(x_test)

# print total missing
print('Total missing values after imputation: %d' % sum(isnan(x_test).flatten()))

Total missing values before imputation: 467
Total missing values after imputation: 0


In [172]:
x_test = pd.DataFrame(x_test)

#Compruebo que no queda ningún NaN en la base de datos. De esa manera ya puedo trabajar con ella.
check_for_nan = x_test.isnull().values.any()
print (check_for_nan)

False


In [173]:
x_test.columns = x_train.columns

In [174]:
"""x_train.columns = training.columns
['Código Externo', 'Edad del Paciente', 'Sexo del Paciente', 'LEU',
       'NEU', 'NEUp', 'LIN', 'LINp', 'MON', 'MONp', 'EOS', 'EOSp', 'BAS',
       'BASp', 'IG', 'IGp', 'ERIT', 'HGB', 'HCT', 'VCM', 'HCM', 'CHCM', 'RDW',
       'ERBL', 'PLTI', 'VPM', 'rNe/L', 'rPL/L', 'MacR', 'MicR', 'BA-D#',
       'NEFSC', 'BA-D%', 'BA-N#', 'BA-N%', 'HFLCA', 'HFLCP', 'LY-WX', 'LY-WY',
       'LY-WZ', 'LY-X', 'LY-Y', 'LY-Z', 'MO-WX', 'MO-WY', 'MO-WZ', 'MO-X',
       'MO-Y', 'MO-Z', 'NE-WX', 'NE-WY', 'NE-WZ', 'ASLYP', 'ASLYA', 'RELPL',
       'RELYP', 'RELYA', 'NEUGI', 'NEURI', 'ASLPL', 'Revision']"""

"x_train.columns = training.columns\n['Código Externo', 'Edad del Paciente', 'Sexo del Paciente', 'LEU',\n       'NEU', 'NEUp', 'LIN', 'LINp', 'MON', 'MONp', 'EOS', 'EOSp', 'BAS',\n       'BASp', 'IG', 'IGp', 'ERIT', 'HGB', 'HCT', 'VCM', 'HCM', 'CHCM', 'RDW',\n       'ERBL', 'PLTI', 'VPM', 'rNe/L', 'rPL/L', 'MacR', 'MicR', 'BA-D#',\n       'NEFSC', 'BA-D%', 'BA-N#', 'BA-N%', 'HFLCA', 'HFLCP', 'LY-WX', 'LY-WY',\n       'LY-WZ', 'LY-X', 'LY-Y', 'LY-Z', 'MO-WX', 'MO-WY', 'MO-WZ', 'MO-X',\n       'MO-Y', 'MO-Z', 'NE-WX', 'NE-WY', 'NE-WZ', 'ASLYP', 'ASLYA', 'RELPL',\n       'RELYP', 'RELYA', 'NEUGI', 'NEURI', 'ASLPL', 'Revision']"

# 3. XGBoost

Comienzo seleccionando únicamente los parámetros que elgió el método XGBoost (Revisar los parámetros seleccionados por los métodos cuando Javier me responda a la eliminación de features muy correlacionadas)

In [200]:
### AHORA TENGO QUE PONER AQUI LAS VARIABLES QUE HA ELEGIDO FINALMENTE EL MODELO XGBOOST
x_train_XGB = x_train_aux[['Código Externo', 'IGp','NEU','MO-Y','rNe/L','VPM','IG','HCM','ERIT','BA-N#','Revision']].copy()
x_test_XGB = x_test[['Código Externo', 'IGp','NEU','MO-Y','rNe/L','VPM','IG','HCM','ERIT','BA-N#','Revision']].copy()

**Jorge** antes de comenzar a realizar la parte de regresión lineal hay que hacer exactamente lo mismo que hicimos con el dataframe original, es decir el tratamiento de outliers y la imputación. Esta parte tiene que ser exactamente igual que antes. Es decir, será necesario utilizar todas las variables, porque creo que utilizábamos KNN imputer. 

Cuando tengas todos los valores sin NaN, es cuando podremos comenzar a realizar la parte de incluir la tendencia temporal. 


Esto ya lo tienes hecho, por lo que no debería tomarte mucho tiempo. Creoq ue la secuencia de pasos debería ser:

1. no quitar las variables seleccionadas
2. incluir revisión y label
3. separar en training y test (utiliando el identificador, para que un paciente esté en training o test, pero no en ambos)
4. identificación de outliers
5. imputación de outlires con knn (exactamente el mismo esquema que has hecho antes)
6. Selección de las variables que se han elegido con xgboost.
7. Incorporar coeficintes de la regresión lineal para capturar la dinámica.

### 2.2.2 Búsqueda de coeficientes (REGRESION LINEAL)

Vamos a realizar un análisis de tendencias para cada variable de forma poblacional. También usaremos este análisis de tendencias de forma individual para cada paciente, de forma que se pueda incorporar la información dinámica que genera dentro del modelo de predicción basado en machine learning.

Vamos a generar la función que obtiene el modelo de regresión lineal. El modelo que planteamos es un polinomio de segundo orden:

$y = a + bx + cx^2$ ;

donde la x corresponde al número de revisión.

Ahora debo de conseguir introducir 3 coeficientes por cada una de los parámetros seleccionados por el método y por cada uno de los pacientes

In [201]:
from sklearn.linear_model import LinearRegression

def lin_reg_model(revi,x_val):
    """
    Función que ajusta un modelo de regresión lineal para el conjunto de datos que viene dado por rev y x_val. Vamos a usar
    sklearn, pero podríamos utilizar cualquier otro módulo de algebra lineal
    """
    
    #generamos la matrix X, que tiene que contener en una columna rev y en otra rev**2
    
    
    rev_2 = revi**2
    
    X = np.vstack((np.ones(revi.shape),revi,rev_2)).T #JA Como quiero un polinomio de segundo orden... meto
                      #unos que me sirven como "a", rev que me sirve como "b" y rev_2 que me sirve como "c"
    
    #ajustamos el modelo de regresion lineal
    
    #le digo false, para que me devuelva todo los coeficientes en coef_
    clf = LinearRegression(fit_intercept=False) #yo creo que lo que haría, para que fuesen todos comparables sería 
    # normalizar los datos pero no lo tengo claro, lo tendría que pensar
    
    clf.fit(X,x_val)
    
    #devolvemos los coeficientes
    return clf.coef_

In [202]:
#Calculo de medias según el dataframe y la variable

def mean_review(dataframe,variable):
    mean_array = []
    for i in dataframe.columns[1:10]:    ###NO LO CLARO
        df_mask=dataframe['Revision']==i
        filtered_df = dataframe[df_mask]

        mean_df = filtered_df[variable].mean()  
        #si tenemos valores que son NaN nos va a dar ese error
        mean_array.append(mean_df)
 
    return mean_array,filtered_df #Tengo que poner return en lugar de print, pues si pongo print cuando ejecuto la funcion me sale None junto a los valores. Y si me 
              #None, no puedo meter la función en lin_reg_model.

In [203]:
"""
revi= np.array([1,2,3,4])
X = []

for i in cd_train:  
      df_pat = x_train.loc[x_train["Código Externo"] == i]
      
        for j in df_pat.columns.values (:,2:9):
             coef_pat = lin_reg_model(revi,df_pat)
"""
x_train_XGB["Código Externo"] = x_train_XGB["Código Externo"].astype(int)

In [204]:
x_train_XGB[x_train_XGB["Código Externo"] == 126455]


,Código Externo,IGp,NEU,MO-Y,rNe/L,VPM,IG,HCM,ERIT,BA-N#,Revision
0,126455,1.80,11.880,109.70,6.750,11.30,0.260,28.10,4.700,0.030,1.0
1,126455,1.34,6.968,117.06,2.776,11.12,0.126,29.54,4.486,0.024,2.0
2,126455,1.20,11.730,120.60,3.262,11.20,0.150,28.60,4.680,0.010,3.0
3,126455,6.30,7.900,115.60,3.322,10.30,0.660,28.20,5.630,0.050,4.0


In [320]:
X = [] #final matrix with rows = one per patient, columns = variables valuest at review 1 and temporal evolution


#run over each codigo externo => each patient
for i,pat in enumerate(cd_train):
    cols_name = []
    #print(i)
    x_aux = [] #individual row in the X matrix for a particular pat
    
    #create a mini df with for reviews
    #print(x_train_XGB["Código Externo"])
    df_pat = x_train_XGB[x_train_XGB["Código Externo"] == pat].copy()
    
    #order mini df according to Revision
    df_pat.sort_values(by=["Revision"],inplace= True)

    #print(df_pat)
    
    #for each variable
    feat_name = df_pat.columns[1:-1]
    #print(feat_name)
    
    for var in feat_name:
        #get the value for the first review
        var_value_1 = df_pat[var][df_pat["Revision"] == 1]
        #print(var_value_1.values)
        if var_value_1.values.size == 0:
            #there are some blank rows, we just skip them
            #print(df_pat[["Código Externo",var]])
            continue
            
        #store the value in the x_aux    
        x_aux.append(var_value_1.values[0])
        
        #create the cols_name for the first pat only
        if pat == "126455":
            cols_name.append(var)
        
        #compute the linear regression model
        
        #review values for this var
        var_review_values = df_pat[var].values
        
        #compute the actual coefficientes for the linear regresssion
        revi = np.arange(1,len(var_review_values)+1)
        coefs = lin_reg_model(revi,var_review_values)
        
        """
        plt.figure()
        plt.plot(revi,var_review_values)
        xx = np.linspace(1,4,20)

        XX = np.vstack((np.ones(xx.shape),xx,xx**2)).T
        plt.plot(xx,XX.dot(coefs),'k--',label = 'real values')
        plt.xlabel(var)
        """
        
        #store coefficients
        #x_aux.append()
        #print(len(coefs))
        for c in coefs:
            x_aux.append(c)
            
        cols_name.append(var)
        cols_name.append(var+'_1')
        cols_name.append(var+'_2')
        cols_name.append(var+'_3')
        
        #stick x_aux in X
        
    print(len(x_aux))
    
    if len(x_aux)==0:
        continue
        
    X.append(x_aux)
    

36
36
36
36
36
36
0
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
0
36
36
36
36
36
36
36
36
36
36
36
0
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
36
0
36
36
36
36
36
36
36


In [316]:
XX = np.array(X)

## 2.3 GBoost

In [318]:
print(XX)

[[ 1.80000000e+00  6.27000000e+00 -5.61400000e+00 ...  7.45000000e-02
  -5.29000000e-02  1.15000000e-02]
 [ 7.00000000e-01  2.14285714e-01  2.28571429e-01 ...  1.42857143e-03
   2.85714286e-03  5.71428571e-03]
 [ 4.20000000e+00  8.45000000e+00 -5.72000000e+00 ...  1.00000000e-02
   8.72132228e-18 -1.63497688e-18]
 ...
 [ 7.00000000e-01  1.50000000e-01  6.00000000e-01 ...  7.05000000e-02
  -6.39000000e-02  1.45000000e-02]
 [ 9.00000000e-01  4.48500000e+00 -3.95700000e+00 ...  4.35000000e-02
  -3.57000000e-02  9.50000000e-03]
 [ 3.00000000e-01 -5.00000000e-01  9.40000000e-01 ...  3.70146075e-17
   1.90000000e-02 -5.00000000e-03]]
